<a href="https://colab.research.google.com/github/The237/ANN_Train/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np

In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [5]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data/PlantVillage",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE
)

Found 2103 files belonging to 4 classes.


In [6]:
class_names = dataset.class_names

In [7]:
n_classes = len(class_names)

In [8]:
def get_dataset_partitions_tf(ds, train_split = .8, val_split = .1, test_split = .1, shuffle = True, shuffle_size = 10000):

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)

    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [9]:
train_ds, val_ds, test_ds =  get_dataset_partitions_tf(dataset)

In [10]:
# improve program performances by putting images in cache
train_ds = train_ds.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [11]:
# layers to
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [12]:
# layers for
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(.2)
])

In [13]:
input_shape = (BATCH_SIZE, IMAGE_SIZE , IMAGE_SIZE, CHANNELS)

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = input_shape),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size = (3,3) ,activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size = (3,3) ,activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size = (3,3),activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size = (3,3),activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation = "relu"),
    layers.Dense(n_classes, activation = "softmax")
])

model.build(input_shape = input_shape)

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                  

In [15]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

In [16]:
history = model.fit(
    train_ds,
    epochs= EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data = val_ds
)

Epoch 1/50
52/52 [==============================] - 278s 5s/step - loss: 0.8508 - accuracy: 0.5727 - val_loss: 0.5761 - val_accuracy: 0.7812
Epoch 2/50
52/52 [==============================] - 260s 5s/step - loss: 0.4045 - accuracy: 0.8329 - val_loss: 0.5482 - val_accuracy: 0.7760
Epoch 3/50
52/52 [==============================] - 267s 5s/step - loss: 0.2611 - accuracy: 0.8918 - val_loss: 0.2544 - val_accuracy: 0.8958
Epoch 4/50
52/52 [==============================] - 261s 5s/step - loss: 0.2326 - accuracy: 0.9062 - val_loss: 0.3810 - val_accuracy: 0.8438
Epoch 5/50
52/52 [==============================] - 269s 5s/step - loss: 0.2352 - accuracy: 0.9129 - val_loss: 0.1804 - val_accuracy: 0.9219
Epoch 6/50
52/52 [==============================] - 259s 5s/step - loss: 0.1633 - accuracy: 0.9381 - val_loss: 0.2034 - val_accuracy: 0.9271
Epoch 7/50
52/52 [==============================] - 263s 5s/step - loss: 0.1466 - accuracy: 0.9525 - val_loss: 0.3218 - val_accuracy: 0.8542
Epoch 8/50
52

In [19]:
# Save the model
import os
model_version = max([int(i) for i in os.listdir("models") + [0]]) + 1
model.save(f"models/{model_version}")

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
%cd /content/drive/MyDrive/models

/content/drive/MyDrive/models


In [27]:
!zip -r /content/models.zip /content/models/

  adding: content/models/ (stored 0%)
  adding: content/models/1/ (stored 0%)
  adding: content/models/1/variables/ (stored 0%)
  adding: content/models/1/variables/variables.index (deflated 67%)
  adding: content/models/1/variables/variables.data-00000-of-00001 (deflated 11%)
  adding: content/models/1/saved_model.pb (deflated 89%)
  adding: content/models/1/assets/ (stored 0%)
  adding: content/models/1/fingerprint.pb (stored 0%)
  adding: content/models/1/keras_metadata.pb (deflated 93%)
